<a href="https://colab.research.google.com/github/arnavjain2710/IITISoC24-ML-36/blob/main/IITISOC_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pip dependencies ek baar install karke session restart karlena, uske baad model interrupr kare bhi toh restart matt karna session galti se bhi

In [ ]:
!pip install ffmpeg-python
!pip install ultralytics
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.2/800.2 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Saare functions aur model loading, u need not run it again and again

In [ ]:
from IPython.display import HTML
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    mimeType : 'audio/webm;codecs=opus'
  };
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
    }
  };
  recorder.start();
};

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... please wait!"
  }
}

function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve => {
  recordButton.onclick = () => {
    toggleRecording()

    sleep(2000).then(() => {
      resolve(base64data.toString())
    });
  }
});
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('output.wav', format='wav')  # Save as output.wav
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, _ = process.communicate(input=binary)

  # Read the saved .wav file
  sr, audio = wav_read('output.wav')

  return audio, sr

#import pandas as pd
import librosa
import matplotlib.pyplot as plt
import os
#import numpy as np

def create_mels(audio , image_file):
  fig = plt.figure(figsize=[1, 1])
  ax = fig.add_subplot(111)
  ax.axes.set_axis_off()
  fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
  y, sr = librosa.load(audio, sr=16000)
  mels = librosa.feature.melspectrogram(y=y, sr=sr)
  log_mels = librosa.power_to_db(mels, ref=np.max)
  librosa.display.specshow(log_mels, sr=sr, cmap='viridis')
  fig.savefig(image_file, dpi=400, bbox_inches='tight', pad_inches=0)
  plt.close(fig)
def create_png(input_path , output_path):
        input_file = input_path
        output_file = os.path.join(output_path, os.path.basename(input_file).replace('.wav', '.png'))
        create_mels(input_file, output_file)

from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image

model_path = '/content/best.pt'

# Load the YOLO model
model_emo = YOLO(model_path)

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline



device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

from transformers import BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',  # Choose from 'fp4' or 'nf4'
    bnb_4bit_compute_dtype=torch.float16  # Can be 'float16', 'bfloat16', 'float32', etc.
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
messages = [
    {"role": "system", "content": "You are an assistant chatbot who continues the conversation understanding the user's emotion."},
]

def add_message(role, content):
    messages.append({"role": role, "content": content})





WARNING ⚠️ /content/best.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 12.1s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
!pip install TTS


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 78.6 MB/s eta 0:00:00
  Preparing metadata (setup.py

In [ ]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/usr/local/lib/python3.10/dist-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:29<00:00, 79.1MiB/s]
100%|██████████| 1.87G/1.87G [00:30<00:00, 61.8MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 12.4kiB/s]
 50%|█████     | 181k/361k [00:00<00:00, 1.45MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 735kiB/s] 
100%|██████████| 32.0/32.0 [00:00<00:00, 55.9iB/s]
 12%|█▏        | 949k/7.75M [00:00<00:00, 9.48MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:14<00:00, 9.48MiB/s]

Isko baar baar run kr skte

In [ ]:
audio, sr = get_audio()
import scipy.io.wavfile as wav
from scipy.signal import resample
import numpy as np

sampling_rate, data = wav.read("/content/output.wav")

new_sampling_rate = 16000

num_samples = int(len(data) * new_sampling_rate / sampling_rate)

resampled_data = resample(data, num_samples)

wav.write("resampled_audio.wav", new_sampling_rate, np.asarray(resampled_data, dtype=np.int16))

create_png('/content/resampled_audio.wav' , '/content')
audio_image_path='/content/resampled_audio.png'
test_image_path = audio_image_path
results = model_emo.predict(source=test_image_path)

predicted_class = results[0].names[results[0].probs.top1]
confidence = results[0].probs.top1conf

#print(f'Predicted Class: {predicted_class}, Confidence: {confidence:.2f}')

# Display the test image, can be removed, no need
img = Image.open(test_image_path)
plt.imshow(img)
plt.axis('off')
#plt.show()
emotion=predicted_class

sample1="/content/resampled_audio.wav"

result1 = pipe(sample1)
#print(result1)
#print(result1["text"])
prompt="text : " + result1["text"] + "emotion of user : " + emotion

add_message( "user" , prompt)
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
#print(tokenizer.decode(response, skip_special_tokens=True))
add_message("model" , tokenizer.decode(response, skip_special_tokens=True))

tts.tts_to_file(#text=tokenizer.decode(response, skip_special_tokens=True),
                text=tokenizer.decode(response, skip_special_tokens=True),
                file_path="output.wav",
                speaker_wav="/content/Mixofemotions.wav",
                language="en")
import librosa
import IPython.display as ipd
file_path = '/content/output.wav'
speech, sr = librosa.load(file_path, sr=None)

# Display the audio player
ipd.Audio(speech, rate=sr)


image 1/1 /content/resampled_audio.png: 640x640 happy 0.95, sad 0.04, neutral 0.01, anger 0.00, fear 0.00, 11.6ms
Speed: 35.9ms preprocess, 11.6ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


 > Text splitted to sentences.
["Whoa, that's a surprising turn of events!", "I'm glad to hear that your boss is willing to give you a second chance, though!", "It sounds like you're in a bit of a tricky situation, but at least you have a chance to turn things around.", "What's your plan of action going to be?", 'Are you going to take on the extra work or try to negotiate a different solution with your boss?']
 > Processing time: 28.279181003570557
 > Real-time factor: 1.1228282670358027


To start a fresh conversation to make sure model forgets context, run this

In [ ]:
messages = [
    {"role": "system", "content": "You are an assistant chatbot who continues the conversation understanding the user's emotion."},
]